In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import gymnasium as gym
from gymnasium.wrappers import GrayscaleObservation, ResizeObservation, RecordEpisodeStatistics, RecordVideo, TimeLimit
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
import os
import gc
from eval import *
from custom_cr import EnhancedCarRacing
import pandas as pd

### 3.1. Task Specific Metrics

### 3.2. Robustness and Adaptability

1. Robustness to Observation Noise:
   - It runs `num_episodes` episodes with added Gaussian noise to the observations.
   - For each step, it adds noise to the observation before predicting an action.
   - It accumulates the total reward for each episode and stores it in `noise_rewards`.

2. Robustness to Environment Perturbations:
   - It runs another set of episodes, this time applying random perturbations to the environment.
   - With probability `perturbation_prob`, it adds uniform random noise to the observation.
   - It accumulates the total reward for each episode and stores it in `perturbation_rewards`.

3. Results Computation and Output:
   - For each robustness scenario, it calculates and prints the mean and standard deviation of the rewards.
   
This function is designed to evaluate how well the trained model performs under different types of perturbations and variations, which is crucial for assessing the robustness and generalization capabilities of the reinforcement learning agent.

In [3]:
def evaluate_robustness(model, env, num_episodes=10, noise_std=0.1, perturbation_prob=0.1):
    """
    Evaluate the robustness of a trained model under various challenging conditions.

    This function tests the model's ability to handle noisy observations, random perturbations, 
    and diverse initial states in the environment. Results include performance metrics such as 
    mean rewards and standard deviations under each condition.

    Args:
        model (BaseAlgorithm): ThKeysView(NpzFile './best_model/best_model_2.1.1.zip' with keys: data, pytorch_variables.pth, policy.pth, policy.optimizer.pth, _stable_baselines3_version...)e trained model to evaluate. Should support `.predict()` for action selection.
        env (gym.Env): The environment in which the model will be tested.
        num_episodes (int, optional): The number of episodes to run for each robustness scenario. Defaults to 10.
        noise_std (float, optional): Standard deviation of Gaussian noise added to observations. Defaults to 0.1.
        perturbation_prob (float, optional): Probability of applying random perturbations to observations. Defaults to 0.1.

    Returns:
        dict: A dictionary with keys:
            - "noise_rewards": List of total rewards for episodes with noisy observations.
            - "perturbation_rewards": List of total rewards for episodes with random perturbations.
            - "initial_state_rewards": List of total rewards for episodes starting from diverse initial states.
        Each list includes rewards from `num_episodes` episodes.
    """
    results = {
        "noise_rewards": [],
        "perturbation_rewards": []
    }
    
    # Evaluate robustness to observation noise
    print("Evaluating robustness to observation noise...")
    for _ in range(num_episodes):
        print(f"Running episode {_}")
        obs, _ = env.reset()
        total_reward = 0
        done = False
        while not done:
            # Add Gaussian noise to observation
            noisy_obs = obs + np.random.normal(0, noise_std, obs.shape)
            action = model.predict(noisy_obs, deterministic=True)[0]
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
        results["noise_rewards"].append(total_reward)
        print(f"Perturbation episode: Total Reward = {total_reward}")  # For debugging 

    # Evaluate robustness to environment perturbations
    print("Evaluating robustness to environment perturbations...")
    for _ in range(num_episodes):
        print(f"Running episode {_}")
        obs, _ = env.reset()
        total_reward = 0
        done = False
        while not done:
            action = model.predict(obs, deterministic=True)[0]
            
            # Apply random perturbations
            if np.random.random() < perturbation_prob:
                obs = obs + np.random.uniform(-0.5, 0.5, obs.shape)
                
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
        print(f"Perturbation episode: Total Reward = {total_reward}")  # For debugging purposes only, remove in production code.  # Evaluate robustness to environment perturb
        results["perturbation_rewards"].append(total_reward)

    # Compute and return mean and standard deviation for all scenarios
    for key in results:
        rewards = np.array(results[key])
        print(f"{key}: Mean = {rewards.mean()}, Std Dev = {rewards.std()}")
    return results


#### 3.2.1 Baseline - DQN

In [20]:
best_model_baseline_dqn = DQN.load('./models/baseline/DQN/best_model.zip', allow_pickle=True)

/home/inesamorim46/3ano1sem/isia/reinforcement-learning-with-gymnasium/.venv/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(
/home/inesamorim46/3ano1sem/isia/reinforcement-learning-with-gymnasium/.venv/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(
/home/inesamorim46/3ano1sem/isia/reinforcement-learning-with-gymnasium/.venv/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.77GB > 1.04GB
  warnings.warn(


In [21]:
env = gym.make("CarRacing-v3", continuous=False)

In [ ]:
robustness_results_baseline_dqn = evaluate_robustness(best_model_baseline_dqn, env, num_episodes=5)

In [22]:
robustness_results_baseline_dqn = evaluate_robustness(best_model_baseline_dqn, env, num_episodes=1)

Evaluating robustness to observation noise...
Running episode 0
Perturbation episode: Total Reward = 914.0999999999913
Evaluating robustness to environment perturbations...
Running episode 0
Perturbation episode: Total Reward = 907.1999999999932
noise_rewards: Mean = 914.0999999999913, Std Dev = 0.0
perturbation_rewards: Mean = 907.1999999999932, Std Dev = 0.0


In [23]:
robustness_results_baseline_dqn

{'noise_rewards': [914.0999999999913],
 'perturbation_rewards': [907.1999999999932]}

In [39]:
foo = evaluate_robustness(best_model_baseline_dqn, env, num_episodes=1)

Evaluating robustness to observation noise...
Running episode 0
Perturbation episode: Total Reward = 912.0999999999848
Evaluating robustness to environment perturbations...
Running episode 0
Perturbation episode: Total Reward = 835.899999999969
noise_rewards: Mean = 912.0999999999848, Std Dev = 0.0
perturbation_rewards: Mean = 835.899999999969, Std Dev = 0.0


In [40]:
robustness_results_baseline_dqn['noise_rewards'].append(foo['noise_rewards'])
robustness_results_baseline_dqn['perturbation_rewards'].append(foo['perturbation_rewards'])

In [46]:
robustness_results_baseline_dqn = pd.DataFrame(robustness_results_baseline_dqn)
robustness_results_baseline_dqn.to_csv('results/robustness_results_baseline_dqn.csv', index=False)

In [61]:
robustness_results_baseline_dqn = pd.read_csv('results/robustness_results_baseline_dqn.csv')
robustness_results_baseline_dqn

,noise_rewards,perturbation_rewards
0,914.1,907.2
1,823.0,718.9
2,844.3,897.7
3,908.7,916.7
4,914.0,905.1
5,912.1,835.9


In [57]:
def clean_and_convert(value):
    if isinstance(value, str):
        value = value.strip('[]')
    return float(value)

In [59]:
robustness_results_baseline_dqn = robustness_results_baseline_dqn.applymap(clean_and_convert)
robustness_results_baseline_dqn

/tmp/ipykernel_398771/4249244110.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  robustness_results_baseline_dqn = robustness_results_baseline_dqn.applymap(clean_and_convert)


,noise_rewards,perturbation_rewards
0,914.1,907.2
1,823.0,718.9
2,844.3,897.7
3,908.7,916.7
4,914.0,905.1
5,912.1,835.9


#### 3.2.2 Baseline - PPO

#### 3.2.3 Custom Environment - DQN

In [4]:
custom_env = EnhancedCarRacing(render_mode="rgb_array")
custom_env = GrayscaleObservation(custom_env, keep_dim=True)

In [5]:
best_model_custom_dqn = DQN.load('./models/custom/DQN/best_model.zip', allow_pickle=True)

[W1218 20:29:59.936213148 NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


In [34]:
robustness_results_custom_dqn = evaluate_robustness(best_model_custom_dqn, custom_env, num_episodes=1)

Evaluating robustness to observation noise...
Running episode 0
Perturbation episode: Total Reward = -7759.953796605264
Evaluating robustness to environment perturbations...
Running episode 0
Perturbation episode: Total Reward = -4434.551418120928
noise_rewards: Mean = -7759.953796605264, Std Dev = 0.0
perturbation_rewards: Mean = -4434.551418120928, Std Dev = 0.0


In [49]:
foo = evaluate_robustness(best_model_custom_dqn, custom_env, num_episodes=5)

Evaluating robustness to observation noise...
Running episode 0
Perturbation episode: Total Reward = -7584.80748308475
Running episode 1
Perturbation episode: Total Reward = -3840.5075875620796
Running episode 2
Perturbation episode: Total Reward = -7460.315858936726
Running episode 3
Perturbation episode: Total Reward = -7861.306102609049
Running episode 4
Perturbation episode: Total Reward = -7541.33890571855
Evaluating robustness to environment perturbations...
Running episode 0
Perturbation episode: Total Reward = -7514.179581865233
Running episode 1
Perturbation episode: Total Reward = -3519.7626820016026
Running episode 2
Perturbation episode: Total Reward = -7947.5827114948215
Running episode 3
Perturbation episode: Total Reward = -9068.230437492533
Running episode 4
Perturbation episode: Total Reward = -4008.980884820603
noise_rewards: Mean = -6857.655187582231, Std Dev = 1514.5869781322078
perturbation_rewards: Mean = -6411.747259534958, Std Dev = 2225.664461851139


In [47]:
robustness_results_custom_dqn['noise_rewards'].append(foo['noise_rewards'])
robustness_results_custom_dqn['perturbation_rewards'].append(foo['perturbation_rewards'])

In [48]:
robustness_results_custom_dqn

{'noise_rewards': [-7759.953796605264,
  [-7858.43471366518],
  [-4357.975981461611],
  [-7663.295497849881],
  [-8118.9301815847275]],
 'perturbation_rewards': [-4434.551418120928,
  [-7241.206349396435],
  [-7743.716855095754],
  [-7901.444085518807],
  [-7828.554151708476]]}

In [32]:
robustness_results_custom_dqn = pd.DataFrame(robustness_results_custom_dqn)
robustness_results_custom_dqn.to_csv('results/robustness_results_custom_dqn.csv', index=False)
robustness_results_custom_dqn = pd.read_csv('results/robustness_results_custom_dqn.csv')
robustness_results_custom_dqn

ValueError: All arrays must be of the same length

#### 3.2.4 Custom Environment - PPO

---

In [ ]:
# Simulated or pre-loaded CSV paths
# Replace these with your actual file paths
baseline_dqn = pd.read_csv('results/robustness_results_baseline_dqn.csv')
baseline_ppo = pd.read_csv('results/robustness_results_baseline_ppo.csv')
custom_dqn = pd.read_csv('results/robustness_results_custom_dqn.csv')
custom_ppo = pd.read_csv('results/robustness_results_custom_ppo.csv')

# Calculate the mean of rewards for each model
noise_means = {
    "Baseline DQN": baseline_dqn["noise_rewards"].mean(),
    "Baseline PPO": baseline_ppo["noise_rewards"].mean(),
    "Custom DQN": custom_dqn["noise_rewards"].mean(),
    "Custom PPO": custom_ppo["noise_rewards"].mean(),
}

perturbation_means = {
    "Baseline DQN": baseline_dqn["perturbation_rewards"].mean(),
    "Baseline PPO": baseline_ppo["perturbation_rewards"].mean(),
    "Custom DQN": custom_dqn["perturbation_rewards"].mean(),
    "Custom PPO": custom_ppo["perturbation_rewards"].mean(),
}

# Plot histogram for noise rewards
plt.figure(figsize=(10, 5))
plt.bar(noise_means.keys(), noise_means.values(), color=['blue', 'green', 'orange', 'red'])
plt.title("Comparison of Mean Noise Rewards")
plt.ylabel("Mean Noise Reward")
plt.xlabel("Models")
plt.show()

# Plot histogram for perturbation rewards
plt.figure(figsize=(10, 5))
plt.bar(perturbation_means.keys(), perturbation_means.values(), color=['blue', 'green', 'orange', 'red'])
plt.title("Comparison of Mean Perturbation Rewards")
plt.ylabel("Mean Perturbation Reward")
plt.xlabel("Models")
plt.show()
